In [1]:
import pandas as pd # 0.23.0
import numpy as np
import requests as r
import time
import urllib3
import json

In [2]:
df = pd.read_csv('INT_linksToChange.csv')
df

,title,type,attribute,id,old_link,new_link
0,Aktuelle Pilot- und Demonstrationsprojekte des...,resources,download_url,ff6a28fe-f691-4ee1-87df-50ef4d8decf6,https://data.geo.admin.ch/ch.bfe.energieforsch...,-
1,Aktuelle Pilot- und Demonstrationsprojekte des...,resources,url,ff6a28fe-f691-4ee1-87df-50ef4d8decf6,https://data.geo.admin.ch/ch.bfe.energieforsch...,-
2,Analyse des schweizerischen Energieverbrauchs ...,dataset,url,cd67848c-9051-44e1-9e72-11c153b46abb,http://www.bfe.admin.ch/ogd3,https://www.bfe.admin.ch/ogd3
3,Analyse des schweizerischen Energieverbrauchs ...,resources,url,d904c440-fdb6-4e98-baff-52f73b1f6a06,http://www.bfe.admin.ch/ogd3,https://www.bfe.admin.ch/ogd3
4,Angebote der Innovationsförderung im Energiebe...,dataset,url,44f9dd93-6bd0-4d56-8650-e472b44b72b0,http://www.bfe.admin.ch/ogd4,https://www.bfe.admin.ch/ogd4
...,...,...,...,...,...,...
122,Windatlas Schweiz: Jahresmittel der modelliert...,resources,download_url,98f52d31-ed69-4634-b95d-2ba786f83cfb,https://data.geo.admin.ch/ch.bfe.windenergie-g...,-
123,Windatlas Schweiz: Jahresmittel der modelliert...,resources,url,98f52d31-ed69-4634-b95d-2ba786f83cfb,https://data.geo.admin.ch/ch.bfe.windenergie-g...,-
124,Wochenstatistik Elektrizitätsbilanz - Erzeugun...,dataset,url,979f37ed-fcb5-4400-802b-be0bb2833ea4,http://www.bfe.admin.ch/ogd51,https://www.bfe.admin.ch/ogd51
125,Wochenstatistik Elektrizitätsbilanz - Erzeugun...,resources,uri,2d524296-9638-4ae5-b623-07b253540f7c,https://ckan.opendata.swiss/dataset/9ab8ccdd-a...,-


Adjust relations manually

In [ ]:
df[df.attribute == 'relations']['new_link'].loc[65]

"[{'url': 'https://www.bfe.admin.ch/potenzial-waermenutzung-gewaesser', 'label': 'Bundesamt für Energie BFE - Detailbeschreibung'}, {'url': 'https://opendata.swiss/de/perma/ogd90@bundesamt-fur-energie-bfe', 'label': 'Potenzialstudien und Publikationen zur Wärmenutzung aus Seen und Grundwasser'}, {'url': 'https://www.geocat.ch/geonetwork/srv/ger/catalog.search#/metadata/f51ab0db-c25c-4211-ab07-365734380066', 'label': 'geocat.ch permalink'}]"

In [4]:
tmp = df[df.attribute != 'relations'].copy()
tmp.iloc[0]['old_link']

'https://data.geo.admin.ch/ch.bfe.energieforschung/gpkg/2056/ch.bfe.energieforschung.zip'

In [6]:
a = tmp[tmp.new_link == '-'].copy()
a.reset_index(drop=True, inplace=True)
a.to_json('ABNAHME_badLinks.json', orient='records')

In [7]:
t = tmp[tmp.new_link != '-'].copy()
len(t)

86

In [ ]:
t.iloc[0]['title']

'Analyse des schweizerischen Energieverbrauchs nach Verwendungszwecken'

In [ ]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

s = r.Session()

proxies = None

private_api_key = 'cc1c806e-1fd9-4180-a6bf-7a5304ca503a'

patch_url = 'https://ckan.ogdch-abnahme.clients.liip.ch/api/3/action/package_patch'

header = {'Authorization': private_api_key}

In [ ]:
b = t[t.title == 'Analyse des schweizerischen Energieverbrauchs nach Verwendungszwecken'].copy()
b

,title,type,attribute,id,old_link,new_link
2,Analyse des schweizerischen Energieverbrauchs ...,dataset,url,cd67848c-9051-44e1-9e72-11c153b46abb,http://www.bfe.admin.ch/ogd3,https://www.bfe.admin.ch/ogd3
3,Analyse des schweizerischen Energieverbrauchs ...,resources,url,d904c440-fdb6-4e98-baff-52f73b1f6a06,http://www.bfe.admin.ch/ogd3,https://www.bfe.admin.ch/ogd3


In [ ]:
for index, row in t.iterrows():

  print(index)

  patch_url = ''

  if row['type'] == 'resources':
    patch_url = 'https://ckan.ogdch-abnahme.clients.liip.ch/api/3/action/resource_patch'
  else:
    patch_url = 'https://ckan.ogdch-abnahme.clients.liip.ch/api/3/action/package_patch'

  
  data = {'id': str(row['id']),
         row['attribute'] : str(row['new_link'])}
    
  x = s.post(patch_url,data=data,proxies=proxies,headers=header,verify=False)

  if x.status_code != 200:
    print(data)
    print(x.status_code)

  print()

2

3

4

5

6

7

8

9

12

16

17

18

19

23

24

25

26

27

28

29

30

33

34

35

36

37
{'id': 'a0726f47-bde7-4be4-9473-bb03872b94c1', 'url': 'https://wms.geo.admin.ch/?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetCapabilities&lang=de'}
409

38
{'id': '236081d0-89c7-46fb-834f-41b4f17b5831', 'download_url': 'https://data.geo.admin.ch/ch.bfe.erneuerbarheizen-mehrfamilienhaeuser/csv/2056/ch.bfe.erneuerbarheizen-mehrfamilienhaeuser.zip'}
409

39
{'id': '0f9dae42-e0d9-4ac8-9901-69d1bd02ee8d', 'download_url': 'https://data.geo.admin.ch/ch.bfe.erneuerbarheizen-mehrfamilienhaeuser/gpkg/2056/ch.bfe.erneuerbarheizen-mehrfamilienhaeuser.zip'}
409

40
{'id': '77e289ec-3184-4682-9ac1-477b00b601ad', 'download_url': 'https://data.geo.admin.ch/ch.bfe.erneuerbarheizen-mehrfamilienhaeuser/xtf/2056/ch.bfe.erneuerbarheizen-mehrfamilienhaeuser.zip'}
409

41
{'id': '236081d0-89c7-46fb-834f-41b4f17b5831', 'url': 'https://data.geo.admin.ch/ch.bfe.erneuerbarheizen-mehrfamilienhaeuser/csv/2056/ch.bfe.erneuerbarh

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
l = [37, 38, 39, 40, 41, 42, 43, 50, 103, 104, 105, 106]
len(l)

12

In [ ]:
k = df.loc[l].copy()
for t in k['title'].unique():
  print(t)

Impulsberatung erneuerbar heizen: EFH und MFH bis 6 Wohneinheiten
Impulsberatung erneuerbar heizen: MFH mit mehr als 6 Wohneinheiten
Kehrichtverbrennungsanlagen (KVA)
Thermische Netze (Nahwärme, Fernwärme, Fernkälte)


In [ ]:
k

,title,type,attribute,id,old_link,new_link
37,Impulsberatung erneuerbar heizen: EFH und MFH ...,resources,url,a0726f47-bde7-4be4-9473-bb03872b94c1,http://wms.geo.admin.ch/?SERVICE=WMS&VERSION=1...,https://wms.geo.admin.ch/?SERVICE=WMS&VERSION=...
38,Impulsberatung erneuerbar heizen: MFH mit mehr...,resources,download_url,236081d0-89c7-46fb-834f-41b4f17b5831,http://data.geo.admin.ch/ch.bfe.erneuerbarheiz...,https://data.geo.admin.ch/ch.bfe.erneuerbarhei...
39,Impulsberatung erneuerbar heizen: MFH mit mehr...,resources,download_url,0f9dae42-e0d9-4ac8-9901-69d1bd02ee8d,http://data.geo.admin.ch/ch.bfe.erneuerbarheiz...,https://data.geo.admin.ch/ch.bfe.erneuerbarhei...
40,Impulsberatung erneuerbar heizen: MFH mit mehr...,resources,download_url,77e289ec-3184-4682-9ac1-477b00b601ad,http://data.geo.admin.ch/ch.bfe.erneuerbarheiz...,https://data.geo.admin.ch/ch.bfe.erneuerbarhei...
41,Impulsberatung erneuerbar heizen: MFH mit mehr...,resources,url,236081d0-89c7-46fb-834f-41b4f17b5831,http://data.geo.admin.ch/ch.bfe.erneuerbarheiz...,https://data.geo.admin.ch/ch.bfe.erneuerbarhei...
42,Impulsberatung erneuerbar heizen: MFH mit mehr...,resources,url,0f9dae42-e0d9-4ac8-9901-69d1bd02ee8d,http://data.geo.admin.ch/ch.bfe.erneuerbarheiz...,https://data.geo.admin.ch/ch.bfe.erneuerbarhei...
43,Impulsberatung erneuerbar heizen: MFH mit mehr...,resources,url,77e289ec-3184-4682-9ac1-477b00b601ad,http://data.geo.admin.ch/ch.bfe.erneuerbarheiz...,https://data.geo.admin.ch/ch.bfe.erneuerbarhei...
50,Kehrichtverbrennungsanlagen (KVA),resources,url,2d1bd014-e772-4996-a28d-28ca4cca7bfe,http://wms.geo.admin.ch/?SERVICE=WMS&VERSION=1...,https://wms.geo.admin.ch/?SERVICE=WMS&VERSION=...
103,"Thermische Netze (Nahwärme, Fernwärme, Fernkälte)",resources,download_url,be78136d-fc98-4447-8a10-96cce2c3f260,http://data.geo.admin.ch/ch.bfe.thermische-net...,https://data.geo.admin.ch/ch.bfe.thermische-ne...
104,"Thermische Netze (Nahwärme, Fernwärme, Fernkälte)",resources,download_url,2b0c3950-a415-43be-a18d-680f92dc71d2,http://data.geo.admin.ch/ch.bfe.thermische-net...,https://data.geo.admin.ch/ch.bfe.thermische-ne...


In [ ]:
patch_url = 'https://ckan.ogdch-abnahme.clients.liip.ch/api/3/action/resource_patch'
for index, row in k.iterrows():

  print(index)
  
  data = {'id': str(row['id']),
         row['attribute'] : str(row['new_link'])}
    
  x = s.post(patch_url,data=data,proxies=proxies,headers=header,verify=False)

  if x.status_code != 200:
    print(data)
    print(x.status_code)

  print()

37
{'id': 'a0726f47-bde7-4be4-9473-bb03872b94c1', 'url': 'https://wms.geo.admin.ch/?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetCapabilities&lang=de'}
409

38
{'id': '236081d0-89c7-46fb-834f-41b4f17b5831', 'download_url': 'https://data.geo.admin.ch/ch.bfe.erneuerbarheizen-mehrfamilienhaeuser/csv/2056/ch.bfe.erneuerbarheizen-mehrfamilienhaeuser.zip'}
409

39
{'id': '0f9dae42-e0d9-4ac8-9901-69d1bd02ee8d', 'download_url': 'https://data.geo.admin.ch/ch.bfe.erneuerbarheizen-mehrfamilienhaeuser/gpkg/2056/ch.bfe.erneuerbarheizen-mehrfamilienhaeuser.zip'}
409

40
{'id': '77e289ec-3184-4682-9ac1-477b00b601ad', 'download_url': 'https://data.geo.admin.ch/ch.bfe.erneuerbarheizen-mehrfamilienhaeuser/xtf/2056/ch.bfe.erneuerbarheizen-mehrfamilienhaeuser.zip'}
409

41
{'id': '236081d0-89c7-46fb-834f-41b4f17b5831', 'url': 'https://data.geo.admin.ch/ch.bfe.erneuerbarheizen-mehrfamilienhaeuser/csv/2056/ch.bfe.erneuerbarheizen-mehrfamilienhaeuser.zip'}
409

42
{'id': '0f9dae42-e0d9-4ac8-9901-69d1bd02ee8d', 'url'